In [1]:
import pandas as pd
import numpy as np
import sys
import os
import bs4
from IPython.display import display_html,clear_output, HTML
import re
from datetime import datetime
import ast
import itertools
from tqdm import tqdm,trange
from file_tools import *
from request_tools import *
from parse_tools import *
from load_tools import *
from model_tools import *

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 30)
pd.set_option('compute.use_numexpr', False)

import warnings
from pandas.errors import PerformanceWarning
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
pd.options.mode.use_inf_as_na = True
warnings.filterwarnings('ignore', category=PerformanceWarning)
warnings.filterwarnings('ignore', category=DataConversionWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)


In [2]:
df = pd.read_csv('./02-data-parsed/players/b/butleji01/pgl_basic_regular/2012.csv')
df

,Rk,G,GP,Date,Boxscore_id,Age,Tm,Tm_id,H/A,Opp,Opp_id,W/L,Pts_diff,GS,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1,NaN,0,2011-12-25,/boxscores/201112250LAL.html,22-102,CHI,/teams/CHI/2012.html,0,LAL,/teams/LAL/2012.html,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,0,2011-12-26,/boxscores/201112260GSW.html,22-103,CHI,/teams/CHI/2012.html,0,GSW,/teams/GSW/2012.html,0,-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,0,2011-12-29,/boxscores/201112290SAC.html,22-106,CHI,/teams/CHI/2012.html,0,SAC,/teams/SAC/2012.html,1,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,0,2011-12-30,/boxscores/201112300LAC.html,22-107,CHI,/teams/CHI/2012.html,0,LAC,/teams/LAC/2012.html,1,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1.0,1,2012-01-01,/boxscores/201201010CHI.html,22-109,CHI,/teams/CHI/2012.html,1,MEM,/teams/MEM/2012.html,1,40,0.0,5.600000,1.0,2.0,0.500,0.0,0.0,NaN,0.0,0.0,NaN,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,-4.0
5,6,2.0,1,2012-01-03,/boxscores/201201030CHI.html,22-111,CHI,/teams/CHI/2012.html,1,ATL,/teams/ATL/2012.html,1,2,0.0,0.550000,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,3.0,1,2012-01-04,/boxscores/201201040DET.html,22-112,CHI,/teams/CHI/2012.html,0,DET,/teams/DET/2012.html,1,16,0.0,0.850000,0.0,0.0,NaN,0.0,0.0,NaN,2.0,2.0,1.000,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,3.0,0.0
7,8,NaN,0,2012-01-06,/boxscores/201201060ORL.html,22-114,CHI,/teams/CHI/2012.html,0,ORL,/teams/ORL/2012.html,1,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,4.0,1,2012-01-07,/boxscores/201201070ATL.html,22-115,CHI,/teams/CHI/2012.html,0,ATL,/teams/ATL/2012.html,0,-15,0.0,12.000000,3.0,3.0,1.000,0.0,0.0,NaN,6.0,6.0,1.000,2.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,12.0,12.8,8.0
9,10,5.0,1,2012-01-09,/boxscores/201201090CHI.html,22-117,CHI,/teams/CHI/2012.html,1,DET,/teams/DET/2012.html,1,24,0.0,2.716667,1.0,1.0,1.000,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.7,2.0


In [4]:
player_dir = '/players/b/butleji01'
year = 2012
played = df[df['GP'].astype(bool)][['G','Boxscore_id','Tm_id','Opp_id','H/A']]
played['G'] = played['G'].astype(int)
# played.insert(0,'Player_id',player_dir+'.html')
# played.insert(1,'Year',year)
played

,G,Boxscore_id,Tm_id,Opp_id,H/A
4,1,/boxscores/201201010CHI.html,/teams/CHI/2012.html,/teams/MEM/2012.html,1
5,2,/boxscores/201201030CHI.html,/teams/CHI/2012.html,/teams/ATL/2012.html,1
6,3,/boxscores/201201040DET.html,/teams/CHI/2012.html,/teams/DET/2012.html,0
8,4,/boxscores/201201070ATL.html,/teams/CHI/2012.html,/teams/ATL/2012.html,0
9,5,/boxscores/201201090CHI.html,/teams/CHI/2012.html,/teams/DET/2012.html,1
11,6,/boxscores/201201110CHI.html,/teams/CHI/2012.html,/teams/WAS/2012.html,1
14,7,/boxscores/201201160MEM.html,/teams/CHI/2012.html,/teams/MEM/2012.html,0
15,8,/boxscores/201201170CHI.html,/teams/CHI/2012.html,/teams/PHO/2012.html,1
16,9,/boxscores/201201200CLE.html,/teams/CHI/2012.html,/teams/CLE/2012.html,0
19,10,/boxscores/201201250CHI.html,/teams/CHI/2012.html,/teams/IND/2012.html,1


In [5]:
offset_dict = {}
for offset in range(0,len(played)):
    played_offset = played.copy().shift(offset,fill_value=-1)
    offset_dict[offset] = played_offset

pd.concat(offset_dict,axis=1)

0                                                        
     G                   Boxscore_id                 Tm_id   
4    1  /boxscores/201201010CHI.html  /teams/CHI/2012.html  \
5    2  /boxscores/201201030CHI.html  /teams/CHI/2012.html   
6    3  /boxscores/201201040DET.html  /teams/CHI/2012.html   
8    4  /boxscores/201201070ATL.html  /teams/CHI/2012.html   
9    5  /boxscores/201201090CHI.html  /teams/CHI/2012.html   
11   6  /boxscores/201201110CHI.html  /teams/CHI/2012.html   
14   7  /boxscores/201201160MEM.html  /teams/CHI/2012.html   
15   8  /boxscores/201201170CHI.html  /teams/CHI/2012.html   
16   9  /boxscores/201201200CLE.html  /teams/CHI/2012.html   
19  10  /boxscores/201201250CHI.html  /teams/CHI/2012.html   
20  11  /boxscores/201201270CHI.html  /teams/CHI/2012.html   
22  12  /boxscores/201201300WAS.html  /teams/CHI/2012.html   
23  13  /boxscores/201202010PHI.html  /teams/CHI/2012.html   
24  14  /boxscores/201202020NYK.html  /teams/CHI/2012.html   
25  15  /boxscores/201202040MIL.html  /teams/CHI/2012.html   
26  16  /boxscores/201202060NJN.html  /teams/CHI/2012.html   
27  17  /boxscores/201202080NOH.html  /teams/CHI/2012.html   
28  18  /boxscores/201202100CHA.html  /teams/CHI/2012.html   
32  19  /boxscores/201202180CHI.html  /teams/CHI/2012.html   
34  20  /boxscores/201202220CHI.html  /teams/CHI/2012.html   
37  21  /boxscores/201203020CLE.html  /teams/CHI/2012.html   
39  22  /boxscores/201203050CHI.html  /teams/CHI/2012.html   
41  23  /boxscores/201203080CHI.html  /teams/CHI/2012.html   
42  24  /boxscores/201203100CHI.html  /teams/CHI/2012.html   
43  25  /boxscores/201203120CHI.html  /teams/CHI/2012.html   
44  26  /boxscores/201203140CHI.html  /teams/CHI/2012.html   
45  27  /boxscores/201203160CHI.html  /teams/CHI/2012.html   
46  28  /boxscores/201203170CHI.html  /teams/CHI/2012.html   
47  29  /boxscores/201203190ORL.html  /teams/CHI/2012.html   
49  30  /boxscores/201203240CHI.html  /teams/CHI/2012.html   
50  31  /boxscores/201203260CHI.html  /teams/CHI/2012.html   
51  32  /boxscores/201203280ATL.html  /teams/CHI/2012.html   
53  33  /boxscores/201204010OKC.html  /teams/CHI/2012.html   
54  34  /boxscores/201204020CHI.html  /teams/CHI/2012.html   
55  35  /boxscores/201204050CHI.html  /teams/CHI/2012.html   
56  36  /boxscores/201204080NYK.html  /teams/CHI/2012.html   
57  37  /boxscores/201204100CHI.html  /teams/CHI/2012.html   
58  38  /boxscores/201204120CHI.html  /teams/CHI/2012.html   
59  39  /boxscores/201204150DET.html  /teams/CHI/2012.html   
60  40  /boxscores/201204160CHI.html  /teams/CHI/2012.html   
61  41  /boxscores/201204180CHA.html  /teams/CHI/2012.html   
65  42  /boxscores/201204260CHI.html  /teams/CHI/2012.html   

                              1                                  
                  Opp_id H/A   G                   Boxscore_id   
4   /teams/MEM/2012.html   1  -1                            -1  \
5   /teams/ATL/2012.html   1   1  /boxscores/201201010CHI.html   
6   /teams/DET/2012.html   0   2  /boxscores/201201030CHI.html   
8   /teams/ATL/2012.html   0   3  /boxscores/201201040DET.html   
9   /teams/DET/2012.html   1   4  /boxscores/201201070ATL.html   
11  /teams/WAS/2012.html   1   5  /boxscores/201201090CHI.html   
14  /teams/MEM/2012.html   0   6  /boxscores/201201110CHI.html   
15  /teams/PHO/2012.html   1   7  /boxscores/201201160MEM.html   
16  /teams/CLE/2012.html   0   8  /boxscores/201201170CHI.html   
19  /teams/IND/2012.html   1   9  /boxscores/201201200CLE.html   
20  /teams/MIL/2012.html   1  10  /boxscores/201201250CHI.html   
22  /teams/WAS/2012.html   0  11  /boxscores/201201270CHI.html   
23  /teams/PHI/2012.html   0  12  /boxscores/201201300WAS.html   
24  /teams/NYK/2012.html   0  13  /boxscores/201202010PHI.html   
25  /teams/MIL/2012.html   0  14  /boxscores/201202020NYK.html   
26  /teams/NJN/2012.html   0  15  /boxscores/201202040MIL.html   
27  /teams/NOH/2012.html   0  16  /boxscores/201202060NJN.html   
28  /teams/CHA/2012.h

In [ ]:
def parse_team_previous_games_index(TM_OPP_GM_IDX_DF):
    TM_GM_IDX_DF = TM_OPP_GM_IDX_DF.loc[:,('Team',slice(None))].copy()
    TM_PREV_GM_IDX_DF = []
    for team in set(pd.unique(TM_GM_IDX_DF[('Team','Team_id')])).difference([-1]):
        SRC_TM_GM_IDX_SUB_DF        = TM_GM_IDX_DF.loc[(slice(None),team),:]
        if len(SRC_TM_GM_IDX_SUB_DF) == 0:
            continue
        TRF_TM_GM_IDX_SUB_DF_LIST   = [SRC_TM_GM_IDX_SUB_DF.rename(columns={'Team':f'Team_Curr_Gm'})]
        for i in range(1,len(SRC_TM_GM_IDX_SUB_DF)):
            GM_NO = f'0{i}' if i < 10 else f'{i}'
            TM_GM_IDX_SHFTD_DF_I = SRC_TM_GM_IDX_SUB_DF.copy().shift(i,fill_value=-1) 
            TM_GM_IDX_SHFTD_DF_I.rename(columns={'Team':f'Team_Prev_Gm_{GM_NO}'},inplace=True)
            TRF_TM_GM_IDX_SUB_DF_LIST.append(TM_GM_IDX_SHFTD_DF_I)
        TM_GM_IDX_DF_PER_TM = pd.concat(TRF_TM_GM_IDX_SUB_DF_LIST,axis=1)
        TM_PREV_GM_IDX_DF.append(TM_GM_IDX_DF_PER_TM)

    TM_PREV_GM_IDX_DF = pd.concat(TM_PREV_GM_IDX_DF).fillna(-1).sort_index(axis=0)
    for i,col in enumerate(TM_PREV_GM_IDX_DF.columns):
        if col[1] in ['index','H/A']:
            INT_COL = TM_PREV_GM_IDX_DF.pop(col).astype(int)
            TM_PREV_GM_IDX_DF.insert(i,col,INT_COL)
    return TM_PREV_GM_IDX_DF